In [5]:
import os
import re
from bs4 import BeautifulSoup

 '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Janeiro','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Fevereiro','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Marco','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Abril','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Maio','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Junho', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Julho', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Agosto', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Setembro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Outubro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Novembro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Dezembro'

In [6]:
# Caminho para a pasta de descompactados ASC
pasta_conversa = [ '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Janeiro','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Fevereiro','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Marco','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Abril','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Maio','/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Junho', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Julho', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Agosto', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Setembro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Outubro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Novembro', '/Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Dezembro'
]

def extrair_anexos_arquivo(caminho_arquivo):
    anexos_encontrados = []
    try:
        with open(caminho_arquivo, 'r', encoding='latin-1') as file:
            contents = file.read()
        soup = BeautifulSoup(contents, 'html.parser')

        # Encontrar todos os links para anexos
        links_anexos = soup.find_all('a', href=True)

        for link in links_anexos:
            href = link['href']
            if 'anexos/' in href:
                caminho_anexo_origem = os.path.join(os.path.dirname(caminho_arquivo), href)
                anexos_encontrados.append(caminho_anexo_origem)

    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo {caminho_arquivo}: {str(e)}")

    return anexos_encontrados

def extrair_informacoes_arquivo(caminho_arquivo):
    try:
        with open(caminho_arquivo, 'r', encoding='latin-1') as file:
            contents = file.read()
        soup = BeautifulSoup(contents, 'html.parser')

        info_element = soup.find('p')
        if info_element:
            info_text = info_element.get_text(strip=True)
        else:
            info_text = ""

        mensagens = []
        mensagem_elements = soup.find_all('li', class_=["agente", "contato"])
        for mensagem in mensagem_elements:
            time_element = mensagem.find('span', class_='time')
            if time_element:
                time = time_element.get_text(strip=True)
                mensagem_text_element = mensagem.find('p')
                if mensagem_text_element:
                    mensagem_text = mensagem_text_element.get_text(strip=True)
                    mensagens.append({"horario": time, "mensagem": mensagem_text, 'arquivo': caminho_arquivo})

        return {"info_conversa": info_text, "mensagens": mensagens}
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo {caminho_arquivo}: {str(e)}")
        return {"info_conversa": "", "mensagens": []}

def processar_pasta(pasta_conversas):
    print("Procurando em:", pasta_conversas)
    arquivos_html = [os.path.join(pasta_conversas, arquivo) for arquivo in os.listdir(pasta_conversas) if arquivo.endswith('.html')]
    
    json_data = set()  
    for arquivo_html in arquivos_html:
        encontrou_correspondencia = False  
        
        informacoes_arquivo = extrair_informacoes_arquivo(arquivo_html)
        for mensagem in informacoes_arquivo["mensagens"]:
            texto = mensagem["horario"] + mensagem["mensagem"]
            if re.search(r'DEOCLECIO AUGOSTINHO DE LEMOS|403(\.|)077(\.|)158-09', texto, re.I):
                encontrou_correspondencia = True
                json_data.add(mensagem['arquivo'])
                # Print do arquivo encontrado
                print(mensagem['arquivo'])
                #print("Horário:", mensagem["horario"])
                #print("Mensagem:", mensagem["mensagem"])
                print()
                
                # Listar caminhos dos anexos encontrados apenas nas conversas correspondentes
                anexos_arquivo = extrair_anexos_arquivo(arquivo_html)
                for anexo in anexos_arquivo:
                    print("Caminho do anexo:", anexo)
                
                break  
                
        if encontrou_correspondencia:
            continue  
        
    json_data_new = list(json_data)
    for item in json_data_new:
        print(item)

for pasta in pasta_conversa:
    processar_pasta(pasta)


Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Janeiro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Fevereiro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Marco
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Abril
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Maio
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Junho
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Julho
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Agosto
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Setembro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Outubro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados/2021_2022/2021/Novembro
Procurando em: /Volumes/Backup/ASC_Atendimentos_Descompactados